In [33]:
# the imports and black box of Functions
import math
import numpy as np

a = 1 
b = 0  
m = 0.5 
c = 1 
layer_bias = -0.5


def ians_sigmoid(x, a_a, b_b, m_m, c_c, layer=0):
    return (1/ ((1/a_a) + np.exp(((-x*m_m) + b_b + c_c + layer * layer_bias))))* a_a


def calc_loss(output, desired):
    # can change but not a requirement yet
    return (0.5 * ((output - desired) ** 2))


def d_loss_d_weight_2(n1, n2, desired):
    # check the removal of the layer bias fo rthis one and go back to check the others as well
    return ((n2 - desired) * (n2) * (1-(n2)) * n1) # removedall layer_bias can check layer if correct call

def d_loss_d_weight_1(n1, n2, desired, w2, input):
    # check the removal of the layer bias fo rthis one and go back to check the others as well
    return ((n2 - desired) * (n2) * (1-(n2)) * n1 * w2 * (1- n1) * input)


def d_loss_d_bias_2(n2, desired):
    return (n2 - desired) * (n2 - layer_bias) * (1 - (n2 - layer_bias))

def d_loss_d_bias_1(n1, n2, delta2, desired, a2):
    #delta2 = (n2 - desired) * (n2 - layer_bias) * (1 - (n2 - layer_bias))
    return delta2 * a2 * (n1 - layer_bias) * (1 - (n1 - layer_bias))


def update_param(lr, grad, w):
    w -= lr * grad
    return w

def forward_pass(inputs, x,y,z, a, b):
    global m, c
    total = x + y + z
    delays = []
    total_time_input = 0
    for i in range(0, x):
        sig_r = ians_sigmoid(inputs[i], a[i], b[i], m, c)
        delays.append(sig_r + inputs[i])
        total_time_input += delays[i]

    average_input = total_time_input / x
    total_time_hidden= 0
        
    for i in range(x,(y+x)):
        
        sig_r = ians_sigmoid(average_input, a[i], b[i], m, c, layer=1)
        delays.append(sig_r + average_input)
        print(delays[i])
        total_time_hidden += delays[i]

    average_hidden = total_time_hidden / y
    total_time_output= 0


    for i in range((y+x),(z+y+x)):
        

        sig_r = ians_sigmoid(average_hidden, a[i], b[i], m, c, layer=2)
        delays.append(sig_r + average_hidden)
        total_time_output += delays[i]

    average_output = total_time_output / (z)

    return (delays, average_output)


In [34]:


def run_basic(inputs, x, y, z, a, b, desired, lr):
    """
    inputs: list of length x
    a, b: lists of length x+y+z (weights & biases for each neuron in order: input-layer, hidden-layer, output-layer)
    desired: scalar target
    lr: learning rate
    Mutates a and b in-place.
    Returns: (loss, avg_output)
    """
    global m, c, layer_bias

    # === Forward pass ===
    delays = []
    # input layer
    total_in = 0.0
    for k in range(x):
        E = -m * inputs[k] + b[k] + c + 0 * layer_bias
        u = (1 / a[k]) + np.exp(E)
        f = a[k] / u
        δ = f + inputs[k]
        delays.append(δ)
        total_in += δ
    avg_input = total_in / x

    # hidden layer
    total_h = 0.0
    for j in range(y):
        idx = x + j
        E = -m * avg_input + b[idx] + c + 1 * layer_bias
        u = (1 / a[idx]) + np.exp(E)
        f = a[idx] / u
        δ = f + avg_input
        delays.append(δ)
        total_h += δ
    avg_hidden = total_h / y

    # output layer
    total_o = 0.0
    for ℓ in range(z):
        idx = x + y + ℓ
        E = -m * avg_hidden + b[idx] + c + 2 * layer_bias
        u = (1 / a[idx]) + np.exp(E)
        f = a[idx] / u
        δ = f + avg_hidden
        delays.append(δ)
        total_o += δ
    avg_output = total_o / z

    # compute loss
    loss = calc_loss(avg_output, desired)

    # === Backward pass ===
    N = x + y + z
    grads_a = [0.0] * N
    grads_b = [0.0] * N

    # dL/dO and delta for each output neuron
    dL_dO = avg_output - desired
    for ℓ in range(z):
        idx = x + y + ℓ
        # chain through average
        dL_dδ = dL_dO * (1 / z)

        # derivative of f wrt a[idx], b[idx]
        E = -m * avg_hidden + b[idx] + c + 2 * layer_bias
        expE = np.exp(E)
        u = (1 / a[idx]) + expE

        # ∂f/∂a and ∂f/∂b for f = a/u
        df_da = 1/u + 1/(u * u * a[idx])
        df_db = -a[idx] * expE / (u * u)

        grads_a[idx] = dL_dδ * df_da
        grads_b[idx] = dL_dδ * df_db

    # backprop into avg_hidden
    dL_dh = dL_dO  # because each output δ adds directly to avg_hidden with weight 1/z, summed z times → cancels

    # hidden layer
    for j in range(y):
        idx = x + j
        # each hidden δ contributes equally to avg_hidden
        dL_dδ = dL_dh * (1 / y)

        E = -m * avg_input + b[idx] + c + 1 * layer_bias
        expE = np.exp(E)
        u = (1 / a[idx]) + expE

        df_da = 1/u + 1/(u * u * a[idx])
        df_db = -a[idx] * expE / (u * u)

        grads_a[idx] = dL_dδ * df_da
        grads_b[idx] = dL_dδ * df_db

    # backprop into avg_input
    dL_di = dL_dh  # because sum over hidden then 1/y cancels

    # input layer
    for k in range(x):
        idx = k
        dL_dδ = dL_di * (1 / x)

        E = -m * inputs[k] + b[idx] + c + 0 * layer_bias
        expE = np.exp(E)
        u = (1 / a[idx]) + expE

        df_da = 1/u + 1/(u * u * a[idx])
        df_db = -a[idx] * expE / (u * u)

        grads_a[idx] = dL_dδ * df_da
        grads_b[idx] = dL_dδ * df_db

    # === Parameter update ===
    for i in range(N):
        a[i] -= lr * grads_a[i]
        b[i] -= lr * grads_b[i]

    return loss, avg_output






import random
random.seed(14)
x = 4  # l;1 
inputs = [0.5, 0.8, 0.6, 0.9]
y = 10 # #l2 
z = 3   # l3

a = []
for i in range(x+y+z):
    a.append(random.uniform(0.05, -0.05) + 1.0) 

b = []
for i in range(x+y+z):
    b.append(random.uniform(0.05, -0.05))


m = 0.5
c = 1 

desired = 1.8

for i in range(100):
    print(run_basic(inputs, x,y,z, a, b, desired, 0.1))



(0.09416450476605823, 2.233968903876898)
(0.08297514494611541, 2.2073699668510565)
(0.07332311540797724, 2.182944161485659)
(0.06496207998138327, 2.1604499409942615)
(0.057691481644167306, 2.139680678414794)
(0.05134681461492055, 2.1204584672462894)
(0.04579222122685662, 2.102629216127117)
(0.040914798360642705, 2.0860587294967337)
(0.03662017611845823, 2.0706295479745633)
(0.03282905373554481, 2.05623838016794)
(0.029474463045470727, 2.0427939992894006)
(0.02649959012932553, 2.0302155082931015)
(0.023856028945076418, 2.01843089957731)
(0.021502371982137652, 2.0073758519314033)
(0.019403065849803086, 1.996992719915245)
(0.017527476603360032, 1.9872296803573624)
(0.015849122212711845, 1.9780400079348002)
(0.014345039058539334, 1.9693814574180972)
(0.01299525653167644, 1.9612157345402517)
(0.011782359307364489, 1.9535080408797174)
(0.01069112109718754, 1.946226680856727)
(0.009708196961144052, 1.9393427210954635)
(0.008821863820870236, 1.9328296941265035)
(0.008021800823235272, 1.9266633

In [35]:
s1 = ians_sigmoid(inputs[0], 1, 0, 0.5, 1, layer=0)  + inputs[0]

s2 = ians_sigmoid(inputs[1], 1, 0, 0.5, 1, layer=0) + inputs[1]

s3 = ians_sigmoid((s1 + s2)/2, 1, 0, 0.5, 1, layer=1) + (s1 + s2)/2

s4 = ians_sigmoid(s3, 1, 0, 0.5, 1, layer=2) + s3

print(s1, s2, s3, s4)

0.820821300824607 1.1543436937742046 1.4860303144480338 2.163685150377593


In [39]:
import numpy as np

# 1. Standard sigmoid + derivative
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def sigmoid_deriv(z):
    s = sigmoid(z)
    return s * (1 - s)

class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        # Initialize weights and biases
        self.W1 = np.random.randn(hidden_size, input_size)
        self.b1 = np.zeros((hidden_size, 1))
        self.W2 = np.random.randn(output_size, hidden_size)
        self.b2 = np.zeros((output_size, 1))

    def forward(self, x):
        # Hidden layer
        self.z1 = self.W1 @ x + self.b1
        self.a1 = sigmoid(self.z1)
        # Output layer
        self.z2 = self.W2 @ self.a1 + self.b2
        self.a2 = sigmoid(self.z2)
        return self.a2

    def compute_loss(self, y_hat, y):
        return 0.5 * np.sum((y_hat - y) ** 2)

    def backward(self, x, y):
        # 1) Output error
        delta2 = (self.a2 - y) * sigmoid_deriv(self.z2)
        # 2) Hidden error
        delta1 = (self.W2.T @ delta2) * sigmoid_deriv(self.z1)

        # Parameter gradients
        self.dW2 = delta2 @ self.a1.T
        self.db2 = delta2
        self.dW1 = delta1 @ x.T
        self.db1 = delta1

    def update_params(self, lr):
        # Gradient descent step
        self.W2 -= lr * self.dW2
        self.b2 -= lr * self.db2
        self.W1 -= lr * self.dW1
        self.b1 -= lr * self.db1

    def train_step(self, x, y, lr=0.1):
        y_hat = self.forward(x)
        loss = self.compute_loss(y_hat, y)
        self.backward(x, y)
        self.update_params(lr)
        return loss, y_hat

# Example: X=3 inputs, Y=4 hidden, Z=2 outputs
np.random.seed(42)
model = NeuralNetwork(input_size=3, hidden_size=4, output_size=2)

# Dummy data (column vectors)
x = np.array([[0.8], [0.1], [0.3]])
y = np.array([[1.0], [1.5]])

# Training loop
n_epochs = 10
lr = 0.1

for epoch in range(1, n_epochs + 1):
    loss, y_hat = model.train_step(x, y, lr)
    # Flatten prediction for readability
    prediction = y_hat.flatten()
    print(f"Epoch {epoch:2d} — Loss: {loss:.5f} — Predicted: {prediction}")


Epoch  1 — Loss: 1.38932 — Predicted: [0.05066474 0.12981888]
Epoch  2 — Loss: 1.38057 — Predicted: [0.05147972 0.13564856]
Epoch  3 — Loss: 1.37130 — Predicted: [0.05231687 0.14187985]
Epoch  4 — Loss: 1.36145 — Predicted: [0.05317686 0.14854387]
Epoch  5 — Loss: 1.35101 — Predicted: [0.05406037 0.15567347]
Epoch  6 — Loss: 1.33992 — Predicted: [0.05496807 0.16330303]
Epoch  7 — Loss: 1.32816 — Predicted: [0.05590063 0.17146811]
Epoch  8 — Loss: 1.31569 — Predicted: [0.05685871 0.18020503]
Epoch  9 — Loss: 1.30247 — Predicted: [0.05784296 0.18955019]
Epoch 10 — Loss: 1.28848 — Predicted: [0.05885402 0.19953927]


In [37]:
import numpy as np

# 1. Standard sigmoid + derivative
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def sigmoid_deriv(z):
    s = sigmoid(z)
    return s * (1 - s)

class SNN_Network:
    def __init__(self, input_size, hidden_size, output_size):
        # Initialize weights and biases
        self.W1 = np.random.randn(hidden_size, input_size)
        self.b1 = np.zeros((hidden_size, 1))
        self.W2 = np.random.randn(output_size, hidden_size)
        self.b2 = np.zeros((output_size, 1))

    def forward(self, x):
        # Sum of raw inputs (scalar)
        sum_in = np.sum(x)
        # Hidden layer pre-activation
        self.z1 = self.W1 @ x + self.b1
        # Hidden activation ("spike response")
        self.a1 = sigmoid(self.z1)
        # Delay accumulator: add summed raw input to each hidden neuron
        self.d1 = self.a1 + sum_in
        
        # Sum of delayed-hidden (scalar)
        sum_h = np.sum(self.d1)
        # Output layer pre-activation
        self.z2 = self.W2 @ self.d1 + self.b2
        # Output activation
        self.a2 = sigmoid(self.z2)
        # Delay accumulator for output
        self.d2 = self.a2 + sum_h
        
        return self.d2  # use the delayed output as the final "guess"

    def compute_loss(self, y_pred, y):
        return 0.5 * np.sum((y_pred - y) ** 2)

    def backward(self, x, y):
        # Backprop through the final sigmoid
        delta2 = (self.a2 - y) * sigmoid_deriv(self.z2)
        # Gradients for W2 and b2
        self.dW2 = delta2 @ self.d1.T
        self.db2 = delta2
        
        # Backprop into hidden (through W2 and sigmoid)
        delta1 = (self.W2.T @ delta2) * sigmoid_deriv(self.z1)
        # Gradients for W1 and b1
        self.dW1 = delta1 @ x.T
        self.db1 = delta1

    def update(self, lr):
        self.W2 -= lr * self.dW2
        self.b2 -= lr * self.db2
        self.W1 -= lr * self.dW1
        self.b1 -= lr * self.db1

    def train_step(self, x, y, lr=0.1):
        y_pred = self.forward(x)
        loss = self.compute_loss(y_pred, y)
        self.backward(x, y)
        self.update(lr)
        return loss, y_pred

# Example: X=3 inputs, Y=4 hidden, Z=2 outputs, summing inputs at each layer
np.random.seed(42)
model = SNN_Network(input_size=3, hidden_size=4, output_size=2)

# Dummy data (column vectors)
x = np.array([[0.8], [0.1], [0.3]])
y = np.array([[1.0], [0.0]])

# Training loop
n_epochs = 10
lr = 0.1

for epoch in range(1, n_epochs + 1):
    loss, y_pred = model.train_step(x, y, lr)
    print(f"Epoch {epoch:2d} — Loss: {loss:.5f} — Predicted (delayed): {y_pred.flatten()}")


Epoch  1 — Loss: 49.64083 — Predicted (delayed): [7.52598245 7.52949007]
Epoch  2 — Loss: 49.64068 — Predicted (delayed): [7.52597187 7.52947909]
Epoch  3 — Loss: 49.64053 — Predicted (delayed): [7.52596128 7.52946809]
Epoch  4 — Loss: 49.64038 — Predicted (delayed): [7.52595069 7.52945709]
Epoch  5 — Loss: 49.64023 — Predicted (delayed): [7.52594009 7.52944608]
Epoch  6 — Loss: 49.64007 — Predicted (delayed): [7.52592948 7.52943506]
Epoch  7 — Loss: 49.63992 — Predicted (delayed): [7.52591887 7.52942404]
Epoch  8 — Loss: 49.63977 — Predicted (delayed): [7.52590824 7.529413  ]
Epoch  9 — Loss: 49.63962 — Predicted (delayed): [7.52589762 7.52940197]
Epoch 10 — Loss: 49.63946 — Predicted (delayed): [7.52588698 7.52939092]
